# KLL Sketch Tutorial

## Table of Contents

  * [Overview](#Overview)
  * [Set-up](#Set-up)
  * [Creating a KLL Sketch](#Creating-a-KLL-Sketch)
  * [Querying the sketch](#Querying-the-sketch)
  * [Merging Sketches](#Merging-Sketches)
  * [Serializing Sketches for Transportation](#Serializing-Sketches-for-Transportation)
  * [Using in a Data Cube](#Using-in-a-Data-Cube)

### Overview

This tutorual will focus on the KLL sketch. We will demonstrate how to create and feed data into sketches, and also show an option for moving sketches between systems. We will rely on synthetic data to help us better reason about expected results when visualizing.

### Set-up

This tutorial assumes you have already downloaded and installed the python wrapper for the DataSketches library. See the [DataSketches Downloads](http://datasketches.apache.org/docs/Community/Downloads.html) page for details

In [ ]:
from datasketches import kll_floats_sketch

import base64
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(color_codes=True)

### Creating a KLL Sketch

Sketch creation is simple: As with all the sketches in the library, you simply need to decide on your error tolerance, which determines the maximum size of the sketch. The DataSketches library refers to that value as $k$.

We can get an estimate of the expected error bound (99th percentile) without instantiating anything.

In [ ]:
print(kll_floats_sketch.get_normalized_rank_error(160, False))
print(kll_floats_sketch.get_normalized_rank_error(200, False))

As we can see, the (one-sided) error with $k=160$ is about 1.67% versus 1.33% at $k=200$. For the rest of the examples, we will use $200$. We can now instantiate a sketch.

In [ ]:
k = 200
sk = kll_floats_sketch(k)
print(sk)

The sketch has seen no data so far (N=0) and is consequently storing nothing (Retained items=0). Storage bytes refers to how much space would be required to save the sketch as an array of bytes, which in this case is fairly minimal.
Next, we can add some data.

In [ ]:
sk.update(np.random.exponential(size=150))
print(sk)

We added 150 samples, which is few enough that the sketch is still in exact mode, meaning it is storing everything rather than sampling. To be able to compare the sketch to an exact computation, we will generate new data -- and a lot more of it. We will also create a sketch with a much larger $k$ to demonstrate the effect of increasing the sketch size.

In [ ]:
sk = kll_floats_sketch(k)
sk_large = kll_floats_sketch(10*k)
data = np.random.exponential(size=2**24)
sk.update(data)
sk_large.update(data)
print(sk)
print(sk_large)

Here the sketch is well into sampling territory, having processed nearly 17 million items. We can see that the sketch is retaining only 645 items. The 2676 bytes of storage compares to 64MB for raw data using 4-byte floats. Next we will start querying the sketch to better understand the performance. Even the much larger sketch uses less 24k bytes with fewer than 6000 points.

### Querying the sketch

The median for an exponential distribution is $\frac{ln 2}{\lambda}$, and the default numpy exponential distribution has $\lambda = 1.0$, so the median should be close to $0.693$. Similarly, if we ask for the rank $ln 2$, we should get a value close to $0.5$.

In [ ]:
print(f'Theoretical median             : {np.log(2):.6f}')
print(f'Estimated median, k=200        : {sk.get_quantile(0.5):.6f}')
print(f'Estimated median, k=2000       : {sk_large.get_quantile(0.5):.6f}')
print('')
print(f'Exact Quantile of ln(2)        : 0.5')
print(f'Est. Quantile of ln(2), k=200  : {sk.get_rank(np.log(2)):.6f}')
print(f'Est. Quantile of ln(2), k=2000 : {sk_large.get_rank(np.log(2)):.6f}')

One of the common use cases of a quantiles sketch like KLL is visualizing data with a histogram. We can create one from the sketch easily enough, but for this tutorial we also want to know how well we are doing. Fortunately, we can still compute a histogram on this data directly for comparison.

Note that the sketch returns a PMF while the histogram computes data only for the bins between the provided points and must be converted to a PMF. The sketch also returns a bin containing all the mass less than the minimum provided point. In this case that will always be 0, so we discard it for plotting.

In [ ]:
xmin = 0 # could use sk.get_min_value() but we know the bound here
xmax = sk.get_quantile(0.99995) # this will exclude a little data from the exact distribution
num_splits = 40
step = (xmax - xmin) / num_splits
splits = [xmin + (i*step) for i in range(0, num_splits)]
x = splits.copy()
x.append(xmax)

pmf = sk.get_pmf(splits)[1:]
pmf_large = sk_large.get_pmf(splits)[1:]
exact_pmf = np.histogram(data, bins=x)[0] / sk.get_n()

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
plt.title('PMF, k = 200')
plt.ylabel('Probability')
plt.bar(x=splits, height=pmf, align='edge', width=-.07, color='blue')
plt.bar(x=splits, height=exact_pmf, align='edge', width=.07, color='red')
plt.legend(['KLL, k=200','Exact'])

plt.subplot(2,1,2)
plt.title('PMF, k = 2000')
plt.ylabel('Probability')
plt.bar(x=splits, height=pmf_large, align='edge', width=-.07, color='blue')
plt.bar(x=splits, height=exact_pmf, align='edge', width=.07, color='red')
plt.legend(['KLL, k=2000','Exact'])

The sketch with $k=200$ clearly provides a good approximation. In the case of this exponential distribution, we sometimes observe that there is additional mass near the right edge of the tail compared to the true PMF, although still within the provided error bound with high probability. While this is not problematic given the guarantees of the sketch, certain use cases requiring high precision at extreme quantiles may find it less satisfactory. With the larger $k=2000$ sketch, the accuracy is improved while using much less space thena the raw data, although larger than the smaller-$k$ sketch.

We will eventaully provide what we call a Relative Error Quantiles sketch that will have tighter error bounds as you approach the tail of the distribution, which will be useful if you care primarily about accuray in the tail, but that will require a larger sketch.

### Merging sketches

A single sketch is certainly useful, but the real power of sketches comes from the ability to merge them. Here, we will create two simple sketches to demonstrate. For good measure, we'll use different values of $k$ for the sketches, as well as feed them different numbers of points.

In [ ]:
sk1 = kll_floats_sketch(k)
sk2 = kll_floats_sketch(int(1.5 * k))

data1 = np.random.normal(loc=-2.0, size=2**24)
data2 = np.random.normal(loc=2.0, size=2**25)
sk1.update(data1)
sk2.update(data2)

With the KLL sketch, there is no separate object for unions. We can either create another empty sketch and use that as a merge target or we can merge sketch 2 into sketch 1. Taking the latter approach and plotting the resulting histogram gives us the expected distribution. Note that one sketch has twice as many points as the other.

In [ ]:
sk1.merge(sk2)
print(sk1)

We saved the input data so that we can again compute an exact distribution.

In [ ]:
xmin = sk1.get_min_value()
xmax = sk1.get_max_value()
num_splits = 20
step = (xmax - xmin) / num_splits
splits = [xmin + (i*step) for i in range(0, num_splits)]
x = splits.copy()
x.append(xmax)

pmf = sk1.get_pmf(splits)[1:]
cdf = sk1.get_cdf(splits)[1:]
exact_pmf = (np.histogram(data1, bins=x)[0] + np.histogram(data2, bins=x)[0]) / sk1.get_n()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.bar(x=splits, height=pmf, align='edge', width=-.3, color='blue')
plt.bar(x=splits, height=exact_pmf, align='edge', width=.3, color='red')
plt.legend(['KLL','Exact'])
plt.ylabel('Probability')
plt.title('Merged PMF')

plt.subplot(1,2,2)
plt.bar(x=splits, height=cdf, align='edge', width=-.3, color='blue')
plt.bar(x=splits, height=np.cumsum(exact_pmf), align='edge', width=.3, color='red')
plt.legend(['KLL','Exact'])
plt.ylabel('Probability')
plt.title('Merged CDF')

Notice that we do not need to do anything special to merge the sketches despite the different values of $k$, and the 2:1 relative ratio of weights of the two distributions was preserved despite the input sketch size difference.

### Serializing Sketches for Transportation

Being able to move sketches between platforms is important. One of the useful aspects of the DataSketches library in particular is binary compatibility across languages. While this section will remain within python, sketches serialized from C++- or Java-based systems would work identically.

In this section, we will start by creating a tab-separated file with a handfull
of sketches and then load it in as a dataframe. We will encode each binary sketch image as base64.

To simplify sketch creation, the first step will be to define a simple function to generate a line for the file with the given parameters.

In [ ]:
def generate_sketch(family: str, n: int, mean: float, var: float) -> str:
    sk = kll_floats_sketch(200)
    if (family == 'normal'):
        sk.update(np.random.normal(loc=mean, scale=var, size=n))
    elif (family == 'uniform'):
        b = mean + np.sqrt(3 * var)
        a = 2 * mean - b
        sk.update(np.random.uniform(low=a, high=b, size=n))
    else:
        return None
    sk_b64 = base64.b64encode(sk.serialize()).decode('utf-8')
    return f'{family}\t{n}\t{mean}\t{var}\t{sk_b64}\n'

In [ ]:
filename = 'kll_tutorial.tsv'
with open(filename, 'w') as f:
    f.write('family\tn\tmean\tvariance\tkll\n')
    f.write(generate_sketch('normal', 2**23, -4.0, 0.5))
    f.write(generate_sketch('normal', 2**24,  0.0, 1.0))
    f.write(generate_sketch('normal', 2**25,  2.0, 0.5))
    f.write(generate_sketch('normal', 2**23,  4.0, 0.2))
    f.write(generate_sketch('normal', 2**22, -2.0, 2.0))
    f.write(generate_sketch('uniform', 2**21,  0.5, 1.0/12))
    f.write(generate_sketch('uniform', 2**22,  5.0, 1.0/12))
    f.write(generate_sketch('uniform', 2**20, -0.5, 1.0/3))
    f.write(generate_sketch('uniform', 2**23,  0.0, 4.0/3))
    f.write(generate_sketch('uniform', 2**22, -4.0, 1.0/3))                         

If your system has a *nix shell, you can inspect the resulting file:

In [ ]:
!head -2 {filename}

### Using in a Data Cube

Now that we have our file with 10 sketches, we can use pandas to load them in. To ensure that we load the sketches as useful objects, we need to define a converter function.

In [ ]:
deserialize_kll = lambda x : kll_floats_sketch.deserialize(base64.b64decode(x))

df = pd.read_csv(filename,
                 sep='\t',
                 header=0,
                 dtype={'family':'category', 'n':int, 'mean':float, 'var':float},
                 converters={'kll':deserialize_kll}
                )
print(df)

The sketch column is represented by the string equivalent, which is not very useful for viewing here but does show that the column contains the actual objects.

And finally, we can now perform queries on the results.

In [ ]:
query_result = kll_floats_sketch(10*k)
for sk in df.loc[df['family'] == 'normal'].itertuples(index=False):
    query_result.merge(sk.kll)
print(query_result)


Here we see that the resulting sketch has processed 71 million items (272MB of data) and is summarizing it using 563 items, and can be serialized into only 2352 bytes, which includes some sketch metadata.

Finally, we want to visualize this data. Remember that we have a mixture of 5 Gaussian distributions:

| $\mu$ | $\sigma^2$ | n |
|-----:|----:|---------:|
| -4.0 | 0.5 | $2^{23}$ |
|  0.0 | 1.0 | $2^{24}$ |
|  2.0 | 0.5 | $2^{25}$ |
|  4.0 | 0.2 | $2^{23}$ |
| -2.0 | 2.0 | $2^{22}$ |

In [ ]:
xmin = query_result.get_quantile(0.0005)
xmax = query_result.get_quantile(0.9995)
num_splits = 50
step = (xmax - xmin) / num_splits
splits = [xmin + (i*step) for i in range(0, num_splits)]

pmf = query_result.get_pmf(splits)
x = splits.copy()
x.append(xmax)
plt.figure(figsize=(12,6))
plt.title('PMF')
plt.ylabel('Probability')
plt.bar(x=x, height=pmf, align='edge', width=-0.15)